In [1]:
# inporting our own functions
from modules.TrainTestValGen import init_gens
# @TODO add more to the modules folder to save space 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from _thread import start_new_thread

import tensorflow as tf
from keras import backend as K
from keras import regularizers, optimizers

from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout, MaxPooling2D, BatchNormalization

from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session

C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages

In [2]:
import time
from datetime import timedelta

import math
import os

import scipy.misc
from scipy.stats import itemfreq
from random import sample
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Image manipulation
import PIL.Image
from IPython.display import display

In [3]:
# Load data
BBox_df = pd.read_csv('../BBox_List_2017.csv',dtype=str)
de_df = pd.read_csv('../Data_Entry_2017.csv',dtype=str)
train_filter = pd.read_csv('../train_val_list.txt',dtype=str)
train_filter.columns=['Image_Index']
test_filter = pd.read_csv('../test_list.txt',dtype=str)
test_filter.columns=['Image_Index']

In [4]:
# clean up incorrect ages in the data

bad_ages = de_df[de_df['Patient Age'].astype(int) >= 100].index

df = de_df.drop(bad_ages)

df.drop(columns=['Unnamed: 11','OriginalImagePixelSpacing[x','y]','OriginalImage[Width','Height]'], inplace=True)

df.columns = ['Image_Index', 'Finding_Labels', 'Follow-up', 'Patient ID',
       'Patient_Age', 'Patient_Gender', 'View_Position']

# Remove secondary labels
# @TODO handle labels better
# df['Finding_Labels'] = df['Finding_Labels'].apply(lambda x: x.split('|')[0])

#MAYBE TRY TO PASS A LIST OF INTS FOR THE LABELS IN ONE COLUMN

df['Test_Split'] = df['Finding_Labels'].str.split('|')
# df[['label1','label2','label3','label4','label5','label6','label7','label8','label9']] = pd.DataFrame(df['Test_Split'].values.tolist(),index=df.index)
df.drop(columns=['Finding_Labels','Follow-up','Patient ID'], inplace=True)

# Second one didn't work but worked before?
df.replace(np.nan, 'No Finding', inplace=True)
# df.replace('None','No Finding').head()


# Join to the list of test and training to split the data set
train_df = pd.merge(train_filter,df , on='Image_Index', how='inner')
test_df = pd.merge(test_filter,df , on='Image_Index', how='inner')
df.head()

,Image_Index,Patient_Age,Patient_Gender,View_Position,Test_Split
0,00000001_000.png,58,M,PA,[Cardiomegaly]
1,00000001_001.png,58,M,PA,"[Cardiomegaly, Emphysema]"
2,00000001_002.png,58,M,PA,"[Cardiomegaly, Effusion]"
3,00000002_000.png,81,M,PA,[No Finding]
4,00000003_000.png,81,F,PA,[Hernia]


In [5]:
df.shape

(112104, 5)

In [6]:
train_df.columns

Index(['Image_Index', 'Patient_Age', 'Patient_Gender', 'View_Position',
       'Test_Split'],
      dtype='object')

In [7]:
labels = ['label1',
 'label2',
 'label3',
 'label4',
 'label5',
 'label6',
 'label7',
 'label8',
 'label9']
# labels.reshape(labels.shape[0],1).shape

In [8]:
y_labels=['Patient_Age', 'Patient_Gender', 'View_Position',
       'label1', 'label2', 'label3', 'label4', 'label5', 'label6', 'label7',
       'label8', 'label9']
# y_labels=['Patient_Age','Patient_Gender','View_Position','Test_Split']
# y_labels='Test_Split'

In [15]:
#Size Images 
# @TODO test to see what the pixle size is 
datagen = ImageDataGenerator(rescale=1./255 ,validation_split=0.25)
test_datagen=ImageDataGenerator(rescale=1./255)

# Class Mode
Class_Value = 'categorical'

# Target_size for all
TARGET_SIZE = 270

# How we bin out the batches to run onto the GPU
BATCH_SIZE = 32

In [16]:
# @TODO Get this working an import

#Attach CSV to Images Folder and defline our traiing validation and test
def vaild_gen(batch_fit, Class_Value, TARGET_SIZE):
    global valid_generator
    valid_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        directory="../images/",
        x_col="Image_Index",
        y_col='Test_Split',
        subset="validation",
        class_mode=Class_Value,
        target_size=(TARGET_SIZE,TARGET_SIZE),
        batch_size=batch_fit)
    
def train_gen(batch_fit, Class_Value, TARGET_SIZE):
    global train_generator
    train_generator=datagen.flow_from_dataframe(
        dataframe=train_df,
        directory="../images/",
        x_col="Image_Index",
        y_col='Test_Split',
        subset="training",
        class_mode=Class_Value,
        target_size=(TARGET_SIZE,TARGET_SIZE),
        batch_size=batch_fit)
    
def test_gen(batch_fit, Class_Value, TARGET_SIZE):
    global test_generator
    test_generator=test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory="../images/",
        x_col="Image_Index",
        y_col='Test_Split',
        class_mode=Class_Value,
        target_size=(TARGET_SIZE,TARGET_SIZE),
        batch_size=batch_fit)
    
def init_gens(batch_fit, class_value, target):
    start_new_thread(vaild_gen,(batch_fit, class_value,target,  ))
    start_new_thread(train_gen,(batch_fit, class_value,target, ))
    start_new_thread(test_gen,(batch_fit, class_value,target, ))
    
# Resetting global Memory https://github.com/keras-team/keras/issues/12625
# TODO move to module
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass
    # use the same config as you used to create the session
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tf.Session(config=config))
    

In [17]:
init_gens(BATCH_SIZE ,Class_Value, TARGET_SIZE)

In [18]:
#  test_generator=test_datagen.flow_from_dataframe(
#         dataframe=test_df,
#         directory="../images/",
#         x_col="Image_Index",
#         y_col=y_labels,
#         class_mode=None,
#         target_size=(128,128),
#         batch_size=BATCH_SIZE)

In [19]:
# @TODO read https://medium.com/nanonets/how-to-easily-build-a-dog-breed-image-classification-model-2fd214419cde
# and build a better CNN
#layers
# model = Sequential()
# model.add(Conv2D(32, (3, 3), padding='same',
#                  input_shape=(512,512,2)))
# model.add(Activation('relu'))
# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(128, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(128, (3,3)))

# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Activation('relu'))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(TARGET_SIZE,TARGET_SIZE,3)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(15, activation='softmax'))

In [14]:
STEP_SIZE_VALID= valid_generator.n//valid_generator.batch_size

STEP_SIZE_TRAIN= train_generator.n//train_generator.batch_size

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size


model.compile(optimizers.rmsprop(lr=0.0001),
loss="categorical_crossentropy", metrics=["accuracy"])


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    workers=16,
                    use_multiprocessing=False,
                    epochs=25) 

W0801 05:40:51.398927 17596 deprecation_wrapper.py:119] From C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0801 05:40:51.402928 17596 deprecation_wrapper.py:119] From C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0801 05:40:51.451786 17596 deprecation.py:323] From C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/25
5900/5900 [==============================] - 778s 132ms/step - loss: 2.2963 - acc: 0.5657 - val_loss: 2.0323 - val_acc: 0.6259
Epoch 2/25
5900/5900 [==============================] - 773s 131ms/step - loss: 2.2600 - acc: 0.5657 - val_loss: 2.0525 - val_acc: 0.6258
Epoch 3/25
5900/5900 [==============================] - 774s 131ms/step - loss: 2.2615 - acc: 0.5657 - val_loss: 2.0443 - val_acc: 0.6251
Epoch 4/25
5900/5900 [==============================] - 767s 130ms/step - loss: 2.2626 - acc: 0.5658 - val_loss: 2.0728 - val_acc: 0.6256
Epoch 5/25
5900/5900 [==============================] - 769s 130ms/step - loss: 2.2724 - acc: 0.5664 - val_loss: 2.0704 - val_acc: 0.6256
Epoch 6/25
5900/5900 [==============================] - 779s 132ms/step - loss: 2.2963 - acc: 0.5663 - val_loss: 2.0828 - val_acc: 0.6261
Epoch 7/25
5900/5900 [==============================] - 821s 139ms/step - loss: 2.2994 - acc: 0.5666 - val_loss: 2.0742 - val_acc: 0.6262
Epoch 8/25
5900/5900 [============

In [ ]:
test_df.head()

In [20]:
STEP_SIZE_VALID= valid_generator.n//valid_generator.batch_size

STEP_SIZE_TRAIN= train_generator.n//train_generator.batch_size

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size


model.compile(optimizers.rmsprop(lr=0.0001),
loss="categorical_crossentropy", metrics=["accuracy"])


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    workers=16,
                    use_multiprocessing=False,
                    epochs=25) 

Epoch 1/25


C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 2153 invalid image filename(s) in x_col="Image_Index". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 23438 validated image filenames belonging to 15 classes.


C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 7844 invalid image filename(s) in x_col="Image_Index". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 19666 validated image filenames belonging to 15 classes.
Found 59001 validated image filenames belonging to 15 classes.


ResourceExhaustedError: OOM when allocating tensor with shape[574592,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_1/RMSprop/gradients/dense_3/MatMul_grad/MatMul_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
